In [1]:
pip install simpy networkx

In [2]:
import simpy
import networkx as nx
import unittest
import random

In [7]:
# Define a frame class to represent the frames being transferred in the network
class Frame:
    storage_time_limit = 10  # Maximum allowed storage time
    guard_time = 3  # Guard time for frame integrity
    payload_time = 1
    temporal_frame_length = guard_time + payload_time  # Length of the frame in time units

    def __init__(self, env, source, destination, frame_id):
        self.env = env  # Simulation environment
        self.source = source  # Source node of the frame
        self.destination = destination  # Destination node of the frame
        self.creation_time = env.now  # Timestamp of frame creation
        self.status = 'in transit'  # Status of the frame: 'in transit', 'arrived', 'discarded'
        self.path_traveled = [source]  # Path the frame has traveled
        self.frame_id = frame_id  # Unique identifier for the frame
        self.time_in_storage = 0  # Time spent in storage


    def update_storage_time(self, time_spent):
        self.time_in_storage += time_spent
        if self.time_in_storage > Frame.storage_time_limit:
            self.status = 'discarded'
        print(f"Packet {self.frame_id}: {self.time_in_storage}s in storage")

# Define a network node (e.g., user, router, receiver)
class NetworkNode:
    header_processing = 1

    def __init__(self, env, name, node_type, k=2):
        self.env = env  # Simulation environment
        self.name = name  # Name of the node
        self.node_type = node_type  # Type of node: 'sender', 'router', or 'receiver'
        self.queue = simpy.Store(env)  # Queue to store frames
        self.resource = simpy.Resource(env, capacity=k)  # Resource to handle k simultaneous processes

    def process_frames(self):
        while True:
            frame = yield self.queue.get()  # Get the next frame from the queue
            with self.resource.request() as request:
                yield request  # Wait until the resource is available
                if self.node_type in ['router', 'receiver']:

                    # Simulate header processing delay for routers and receivers
                    processing_delay = 1  # Example processing delay
                    yield self.env.timeout(NetworkNode.header_processing)
                    frame.update_storage_time(NetworkNode.header_processing)

                    if frame.status == 'discarded':
                        print(f"Time {self.env.now}: Frame {frame.frame_id} discarded at {self.name}")
                        continue  # Skip further processing if frame is discarded

                if self.name != frame.destination:
                    # If the current node is not the destination, forward the frame
                    self.env.process(route_frame(self.env, frame, network, self))

                else:
                    # If the current node is the destination, deliver the frame
                    frame.status = 'arrived'
                    print(f"Time {self.env.now}: Frame {frame.frame_id} delivered at {self.name}, from {frame.source} to {frame.destination}")

In [8]:
# Define a function to generate frames at a given source node
def generate_frames(env, source_node, destination, interval, max_packets):
    packet_count = 0  # Counter for the number of packets generated
    while packet_count < max_packets:
        yield env.timeout(interval)  # Wait for the next frame generation interval
        # Create a new frame with example attributes
        frame = Frame(env, source_node.name, destination, packet_count + 1)
        source_node.queue.put(frame)  # Put the frame in the source node's queue
        print(f"\nTime {env.now}: Frame {frame.frame_id} created at {source_node.name}, destined for {destination}")
        frames.append(frame)  # Collect the frame for visualization
        packet_count += 1  # Increment the packet counter

def route_frame(env, frame, network, current_node):
    # Calculate the shortest path from the current node to the destination
    all_paths = list(nx.all_shortest_paths(network, source=current_node.name, target=frame.destination, weight='weight'))
    path = random.choice(all_paths)

    # Determine the next hop in the path
    next_hop = path[1]
    # Get the next node object from the dictionary
    next_node = node_dict[next_hop]

    # Simulate the transmission delay based on the edge weight
    transmission_delay = network[current_node.name][next_hop]['weight']
    yield env.timeout(transmission_delay)
    frame.update_storage_time(transmission_delay)

    if frame.status == 'discarded':
        print(f"Time {env.now}: Frame {frame.frame_id} discarded en route at {current_node.name}")
        return  # Stop forwarding if frame is discarded

    frame.path_traveled.append(next_hop)  # Record the path traveled
    print(f"Time {env.now}: Frame {frame.frame_id} forwarded from {current_node.name} to {next_hop}")
    # Put the frame in the next node's queue
    next_node.queue.put(frame)

# Create a directed graph to represent the network topology
def create_grid_graph(network, rows, cols):
    nodes = []
    for i in range(rows):
        for j in range(cols):
            node = f'Node_{i}_{j}'
            nodes.append(node)
            network.add_node(node)
            # Connect to the right neighbor
            if j < cols - 1:
                network.add_edge(node, f'Node_{i}_{j+1}', weight=1)
                network.add_edge(f'Node_{i}_{j+1}', node, weight=1)
            # Connect to the bottom neighbor
            if i < rows - 1:
                network.add_edge(node, f'Node_{i+1}_{j}', weight=1)
                network.add_edge(f'Node_{i+1}_{j}', node, weight=1)
    return nodes

# Create and initialize the simulation environment
def initialize_simulation(grid_size:int, sender, reciever):
    """
    gridsize=int
    sender = list, coords of sender node [x,y]
    reciever = list, coords of reciever node [x,y]
    """
    global env, network, node_dict, frames
    env = simpy.Environment()
    network = nx.Graph()
    frames = []

    # Create a 3x3 grid of nodes
    nodes = create_grid_graph(network, grid_size, grid_size)

    # Define node types (for simplicity, all nodes are routers except the first and last)
    node_types = {}
    for i in range(grid_size):
        for j in range(grid_size):
            if i == sender[0] and j == sender[1]:
                node_types[f'Node_{i}_{j}'] = 'sender'
            elif i == reciever[0] and j == reciever[1]:
                node_types[f'Node_{i}_{j}'] = 'receiver'
            else:
                node_types[f'Node_{i}_{j}'] = 'router'

    # Create network node objects and store them in a dictionary
    node_dict = {name: NetworkNode(env, name, node_types[name], k=2) for name in nodes}

# Run the simulation
def run_simulation(max_packets, interval, gridsize, sender, reciever):
    initialize_simulation(gridsize, sender, reciever)

    # Start the frame generation process at the sender node, creating frames destined for the receiver node
    env.process(generate_frames(env, node_dict['Node_0_0'], 'Node_2_2', interval=interval, max_packets=max_packets))

    # Start the frame processing process for each node in the network
    for node in node_dict.values():
        env.process(node.process_frames())

    # Run the simulation until all packets are generated and processed
    while len(frames) < max_packets or any(frame.status == 'in transit' for frame in frames):
        env.step()

    print(f"\nSimulation completed after generating and processing {max_packets} packets.")


In [9]:
gridsize = 5
sender = [1, 1]
receiver = [3, 3]
initialize_simulation(gridsize, sender, receiver)
print(node_dict['Node_0_0'].node_type)

router


In [10]:
# Main code to run the simulation

max_packets = 3  # Number of packets to generate and process
interval = 5  # Time interval between packet generations
gridsize = 8
sender = [0,0]
reciever = [2,2]

run_simulation(max_packets, interval, gridsize, sender, reciever)


Time 5: Frame 1 created at Node_0_0, destined for Node_2_2
Packet 1: 1s in storage
Time 6: Frame 1 forwarded from Node_0_0 to Node_0_1
Packet 1: 2s in storage
Packet 1: 3s in storage
Time 8: Frame 1 forwarded from Node_0_1 to Node_1_1
Packet 1: 4s in storage

Time 10: Frame 2 created at Node_0_0, destined for Node_2_2
Packet 1: 5s in storage
Time 10: Frame 1 forwarded from Node_1_1 to Node_1_2
Packet 2: 1s in storage
Time 11: Frame 2 forwarded from Node_0_0 to Node_0_1
Packet 1: 6s in storage
Packet 1: 7s in storage
Time 12: Frame 1 forwarded from Node_1_2 to Node_2_2
Packet 2: 2s in storage
Packet 2: 3s in storage
Time 13: Frame 2 forwarded from Node_0_1 to Node_0_2
Packet 1: 8s in storage
Time 13: Frame 1 delivered at Node_2_2, from Node_0_0 to Node_2_2
Packet 2: 4s in storage

Time 15: Frame 3 created at Node_0_0, destined for Node_2_2
Packet 2: 5s in storage
Time 15: Frame 2 forwarded from Node_0_2 to Node_1_2
Packet 3: 1s in storage
Time 16: Frame 3 forwarded from Node_0_0 to Nod

In [131]:
import unittest

class TestNetworkSimulation(unittest.TestCase):

    def setUp(self):
        self.gridsize = 3
        self.sender = [0, 0]
        self.receiver = [2, 2]
        initialize_simulation(self.gridsize, self.sender, self.receiver)
        self.env = env
        self.network = network
        self.node_dict = node_dict
        self.frames = frames

    def test_frame_creation(self):
        frame = Frame(self.env, 'Node_0_0', 'Node_2_2', 1)
        self.assertEqual(frame.source, 'Node_0_0')
        self.assertEqual(frame.destination, 'Node_2_2')
        self.assertEqual(frame.frame_id, 1)
        self.assertEqual(frame.status, 'in transit')
        self.assertEqual(frame.path_traveled, ['Node_0_0'])

    def test_node_initialization(self):
        node = NetworkNode(self.env, 'Node_1_1', 'router', k=2)
        self.assertEqual(node.name, 'Node_1_1')
        self.assertEqual(node.node_type, 'router')
        self.assertEqual(node.resource.capacity, 2)

    def test_generate_frames(self):
        self.env.process(generate_frames(self.env, self.node_dict['Node_0_0'], 'Node_2_2', interval=1, max_packets=5))
        self.env.run(until=6)
        self.assertEqual(len(self.frames), 5)
        for i, frame in enumerate(self.frames):
            self.assertEqual(frame.frame_id, i + 1)
            self.assertEqual(frame.source, 'Node_0_0')
            self.assertEqual(frame.destination, 'Node_2_2')

    def test_custom_grid_initialization(self):
        gridsize = 5
        sender = [1, 1]
        receiver = [3, 3]
        initialize_simulation(gridsize, sender, receiver)
        self.assertIn('Node_1_1', node_dict)
        print(node_dict['Node_0_0'].node_type)
        self.assertEqual(node_dict['Node_1_1'].node_type, 'sender')
        self.assertIn('Node_3_3', node_dict)
        self.assertEqual(node_dict['Node_3_3'].node_type, 'receiver')

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.798s

OK


router

Time 1: Frame 1 created at Node_0_0, destined for Node_2_2

Time 2: Frame 2 created at Node_0_0, destined for Node_2_2

Time 3: Frame 3 created at Node_0_0, destined for Node_2_2

Time 4: Frame 4 created at Node_0_0, destined for Node_2_2

Time 5: Frame 5 created at Node_0_0, destined for Node_2_2
